In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from collections import deque

class MultivariateHawkesPendulum(gym.Wrapper):
    """
    Pendulum environment with N correlated external disturbance sources.
    The disturbances follow a Multivariate Hawkes Process.
    """
    def __init__(self, env, num_sources=5, history_len=20, decay_rate=0.5):
        super().__init__(env)
        self.num_sources = num_sources
        self.history_len = history_len
        self.decay = decay_rate

        # --- Hawkes Process Parameters ---
        # Base intensity (spontaneous events)
        self.mu = np.random.uniform(0.01, 0.05, size=num_sources)

        # Adjacency Matrix (The Graph Structure!)
        # Alpha[i, j] = Influence of Event j on Event i
        # We make a random sparse graph to make spectral analysis interesting later
        self.adjacency = np.random.uniform(0, 0.3, size=(num_sources, num_sources))
        mask = np.random.rand(num_sources, num_sources) > 0.7
        self.adjacency *= mask
        np.fill_diagonal(self.adjacency, 0.1) # Self-excitation

        # State variables for the process
        self.intensities = np.copy(self.mu)
        self.last_events = np.zeros(num_sources) # Binary: did event happen?

        # History Buffer: Stores raw event logs for the Baseline Agent
        # Shape: (History_Len * Num_Sources)
        self.event_history = deque(maxlen=history_len)
        # Fill with zeros initially
        for _ in range(history_len):
            self.event_history.append(np.zeros(num_sources))

        # Update Observation Space: [State (3)] + [Event History (N*T)]
        base_low = self.env.observation_space.low
        base_high = self.env.observation_space.high

        # Events are 0 or 1, but we allow range for flexibility
        history_low = np.zeros(num_sources * history_len)
        history_high = np.ones(num_sources * history_len)

        self.observation_space = spaces.Box(
            low=np.concatenate([base_low, history_low]),
            high=np.concatenate([base_high, history_high]),
            dtype=np.float32
        )

    def _step_hawkes(self):
        """
        Updates the Multivariate Hawkes Process.
        System Dynamics:
        1. Intensity_t = mu + sum(Alpha * Intensity_{t-1} * decay) + Jump from last event
        """
        # Simple discrete-time decay approximation
        # Past influence decays
        self.intensities = self.mu + (self.intensities - self.mu) * np.exp(-self.decay)

        # Excitation: If event j happened last step, intensity i increases by Alpha[i, j]
        excitation = self.adjacency @ self.last_events
        self.intensities += excitation

        # Stochastic Event Generation (Bernoulli based on intensity)
        # Clip probabilities to [0, 1]
        probs = np.clip(self.intensities, 0, 1)
        current_events = np.random.binomial(1, probs).astype(np.float32)

        self.last_events = current_events
        self.event_history.append(current_events)

        return current_events

    def step(self, action):
        # 1. Evolve the External Process
        events = self._step_hawkes()

        # 2. Disturbance impacts the physical system (Torque)
        # We map the 5 sources to a single scalar torque disturbance
        # e.g., Source 0 pulls left, Source 1 pulls right, etc.
        # Weights: [-1, -0.5, 0, 0.5, 1]
        weights = np.linspace(-2.0, 2.0, self.num_sources)
        external_torque = np.dot(events, weights)

        # 3. Apply action + disturbance to the base environment
        # We intercept the step to inject torque.
        # Since Pendulum step() is closed, we use a trick: modify action
        total_action = action + external_torque
        total_action = np.clip(total_action, -2.0, 2.0) # Clip to env limits

        obs, reward, terminated, truncated, info = self.env.step(total_action)

        # 4. Augment Observation (State + History)
        flat_history = np.array(self.event_history).flatten()
        aug_obs = np.concatenate([obs, flat_history]).astype(np.float32)

        return aug_obs, reward, terminated, truncated, info

    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)

        # Reset Hawkes Process
        self.intensities = np.copy(self.mu)
        self.last_events = np.zeros(self.num_sources)
        for _ in range(self.history_len):
            self.event_history.append(np.zeros(self.num_sources))

        flat_history = np.array(self.event_history).flatten()
        aug_obs = np.concatenate([obs, flat_history]).astype(np.float32)

        return aug_obs, info

# --- Verification Script ---
if __name__ == "__main__":
    # Create the environment
    base_env = gym.make("Pendulum-v1")
    # N=5 sources, History=20 steps.
    # Total Input Dim = 3 (pendulum) + 100 (history) = 103
    env = MultivariateHawkesPendulum(base_env, num_sources=5, history_len=20)

    obs, _ = env.reset()
    print(f"Observation Space Shape: {obs.shape}") # Should be (103,)

    # Run a few steps to see if it crashes
    print("\nRunning simulation...")
    for i in range(5):
        action = env.action_space.sample()
        obs, reward, done, _, _ = env.step(action)

        # Check the last few elements of obs to see events
        current_events = obs[-5:]
        print(f"Step {i}: Torque Disturbances: {current_events}")

Observation Space Shape: (103,)

Running simulation...
Step 0: Torque Disturbances: [0. 0. 0. 0. 0.]
Step 1: Torque Disturbances: [0. 0. 0. 0. 0.]
Step 2: Torque Disturbances: [0. 0. 0. 0. 0.]
Step 3: Torque Disturbances: [0. 0. 0. 0. 0.]
Step 4: Torque Disturbances: [0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [ ]:
!pip install stable-baselines3 shimmy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 10.0 MB/s eta 0:00:00


In [ ]:
import time
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback

# --- Re-paste your Environment Class Here (or import it) ---
# (Assuming MultivariateHawkesPendulum class is defined as before)

def make_env():
    """Helper to create the environment for the agent."""
    base_env = gym.make("Pendulum-v1")
    # We use the same parameters as your test
    return MultivariateHawkesPendulum(base_env, num_sources=5, history_len=20)

if __name__ == "__main__":
    # 1. Create Vectorized Environment (faster training)
    # We create 4 parallel environments to speed up data collection
    vec_env = make_vec_env(make_env, n_envs=4)

    # 2. Define the PPO Agent
    # Policy: MlpPolicy (standard dense neural networks)
    # verbose=1 prints training progress
    model = PPO("MlpPolicy", vec_env, verbose=1, seed=42)

    # 3. Parameter Count Check (The "Systems" Metric)
    # The first layer connects Input(103) -> Hidden(64).
    # That is roughly 103 * 64 = 6,592 parameters just for the first layer!
    policy_params = sum(p.numel() for p in model.policy.parameters() if p.requires_grad)
    print(f"BASELINE AGENT - Total Parameters: {policy_params:,}")

    # 4. Train
    print("Starting Training (Baseline)...")
    start_time = time.time()

    # Train for 50,000 steps (enough for Pendulum to learn basic balance)
    model.learn(total_timesteps=50_000)

    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training Complete. Time taken: {training_time:.2f} seconds")

    # 5. Save
    model.save("baseline_ppo_pendulum")
    print("Model saved as 'baseline_ppo_pendulum.zip'")

    # 6. Quick Evaluation
    print("\nEvaluating Baseline Performance...")
    eval_env = make_env()
    obs, _ = eval_env.reset()
    total_reward = 0

    for _ in range(1000):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, _ = eval_env.step(action)
        total_reward += reward
        if done or truncated:
            obs, _ = eval_env.reset()

    print(f"Average Reward over 1000 steps: {total_reward/1000:.2f}")
    # Note: Pendulum-v1 optimal reward is around -150 to -200 per episode (200 steps).
    # So per step, 0 is perfect, -1 is okay. -8 or lower is failing.

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Using cpu device


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


BASELINE AGENT - Total Parameters: 21,763
Starting Training (Baseline)...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.22e+03 |
| time/              |           |
|    fps             | 3071      |
|    iterations      | 1         |
|    time_elapsed    | 2         |
|    total_timesteps | 8192      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -1.22e+03   |
| time/                   |             |
|    fps                  | 1661        |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.001984795 |
|    clip_fraction        | 0.0118      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.41       |
|    explained_variance   | 0.00416     |
|    learning_rate  

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Average Reward over 1000 steps: -5.24


In [ ]:
import scipy.linalg

class SpectralHawkesPendulum(MultivariateHawkesPendulum):
    """
    Same physical environment, but the observation is compressed using
    Spectral Graph Theory (Graph Fourier Transform).
    """
    def __init__(self, env, num_sources=5, k_eigen=3):
        # Initialize the parent class to get the adjacency/physics
        super().__init__(env, num_sources=num_sources, history_len=1) # History len doesn't matter here

        self.k_eigen = k_eigen

        # --- Spectral Graph Theory Setup ---
        # 1. Laplacian Matrix L = D - A
        # (We use the unnormalized Laplacian for simplicity)
        degrees = np.sum(self.adjacency, axis=1)
        D = np.diag(degrees)
        L = D - self.adjacency

        # 2. Eigen Decomposition
        # We want the eigenvectors corresponding to the *smallest* eigenvalues
        # (Low frequency modes = smooth global trends)
        eigenvalues, eigenvectors = scipy.linalg.eigh(L)

        # 3. Sort and keep top k
        # eigenvectors columns are the vectors
        idx = eigenvalues.argsort()
        self.eigenvectors = eigenvectors[:, idx]
        self.projection_matrix = self.eigenvectors[:, :k_eigen] # Shape (N, k)

        # --- Update Observation Space ---
        # Obs = [Pendulum State (3)] + [Spectral Features (k)]
        base_low = self.env.observation_space.low
        base_high = self.env.observation_space.high

        # Spectral features can range loosely (depends on intensities)
        spec_low = -np.inf * np.ones(k_eigen)
        spec_high = np.inf * np.ones(k_eigen)

        self.observation_space = spaces.Box(
            low=np.concatenate([base_low, spec_low]),
            high=np.concatenate([base_high, spec_high]),
            dtype=np.float32
        )

    def step(self, action):
        # 1. Evolve Physics (Same as parent)
        # We need to manually call _step_hawkes to get intensities,
        # but we can just use the parent step and discard its observation
        _ = self._step_hawkes()

        # Calculate Force (Same as parent)
        weights = np.linspace(-2.0, 2.0, self.num_sources)
        external_torque = np.dot(self.last_events, weights)
        total_action = action + external_torque
        total_action = np.clip(total_action, -2.0, 2.0)

        obs, reward, terminated, truncated, info = self.env.step(total_action)

        # 2. The Spectral Transform (The "Elegant" part)
        # Instead of raw events, we take the current latent intensities
        # signal_t = [intensity_1, intensity_2, ...]
        signal = self.intensities

        # Project onto Graph Basis: GFT = U.T @ signal
        spectral_features = self.projection_matrix.T @ signal

        # 3. Construct Compact Observation
        aug_obs = np.concatenate([obs, spectral_features]).astype(np.float32)

        return aug_obs, reward, terminated, truncated, info

    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        # Reset Hawkes
        self.intensities = np.copy(self.mu)
        self.last_events = np.zeros(self.num_sources)

        # Initial Spectral Features
        spectral_features = self.projection_matrix.T @ self.intensities
        aug_obs = np.concatenate([obs, spectral_features]).astype(np.float32)

        return aug_obs, info

In [ ]:
def make_spectral_env():
    base_env = gym.make("Pendulum-v1")
    # k_eigen=3 means we only pass 3 spectral numbers!
    return SpectralHawkesPendulum(base_env, num_sources=5, k_eigen=3)

if __name__ == "__main__":
    # 1. Create Env
    vec_env_spec = make_vec_env(make_spectral_env, n_envs=4)

    # 2. Create Agent
    model_spec = PPO("MlpPolicy", vec_env_spec, verbose=1, seed=42)

    # 3. Parameter Count (The Systems Win)
    # Input is now 6 (3 state + 3 spectral).
    # First layer is 6 * 64 weights. Tiny!
    spec_params = sum(p.numel() for p in model_spec.policy.parameters() if p.requires_grad)

    print(f"SPECTRAL AGENT - Total Parameters: {spec_params:,}")
    print(f"BASELINE AGENT - Total Parameters: 21,763")
    print(f"REDUCTION: {100 * (1 - spec_params/21763):.2f}% smaller")

    # 4. Train
    print("Starting Training (Spectral)...")
    start_time = time.time()
    model_spec.learn(total_timesteps=50_000)
    print(f"Training Complete. Time: {time.time() - start_time:.2f}s")

    # 5. Evaluate
    print("\nEvaluating Spectral Performance...")
    eval_env = make_spectral_env()
    obs, _ = eval_env.reset()
    total_reward = 0

    for _ in range(1000):
        action, _ = model_spec.predict(obs, deterministic=True)
        obs, reward, done, truncated, _ = eval_env.step(action)
        total_reward += reward
        if done or truncated:
            obs, _ = eval_env.reset()

    print(f"Average Reward over 1000 steps: {total_reward/1000:.2f}")

Using cpu device
SPECTRAL AGENT - Total Parameters: 9,347
BASELINE AGENT - Total Parameters: 21,763
REDUCTION: 57.05% smaller
Starting Training (Spectral)...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -1.2e+03 |
| time/              |          |
|    fps             | 3292     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -1.2e+03    |
| time/                   |             |
|    fps                  | 1757        |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.002176545 |
|    clip_fraction        | 0.0083      